# Dask Example Notebook

This example comes from - https://github.com/dask/dask-examples/blob/master/dataframes/01-data-access.ipynb

# DataFrames: Read and Write Data
     
Dask Dataframes can read and store data in many of the same formats as Pandas dataframes.  In this example we read and write data with the popular CSV and Parquet formats, and discuss best practices when using these formats.

In [9]:
from IPython.display import HTML

HTML('<iframe width="560" height="315" src="https://www.youtube.com/embed/0eEsIA0O1iE?rel=0&amp;controls=0&amp;showinfo=0" frameborder="0" allowfullscreen></iframe>')

/srv/conda/envs/notebook/lib/python3.8/site-packages/IPython/core/display.py:717: UserWarning: Consider using IPython.display.IFrame instead
  warnings.warn("Consider using IPython.display.IFrame instead")


## Start Dask Client for Dashboard

Starting the Dask Client is optional.  It will provide a dashboard which 
is useful to gain insight on the computation.  

The link to the dashboard will become visible when you create the client below.  We recommend having it open on one side of your screen while using your notebook on the other side.  This can take some effort to arrange your windows, but seeing them both at the same is very useful when learning.

In [10]:
from dask.distributed import Client
client = Client(n_workers=1, threads_per_worker=4, processes=False, memory_limit='2GB')
client

/srv/conda/envs/notebook/lib/python3.8/site-packages/distributed/node.py:151: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 40083 instead
  warnings.warn(


Client Scheduler: inproc://10.0.1.91/2366/9 Dashboard: http://10.0.1.91:40083/status,Cluster Workers: 1 Cores: 4 Memory: 2.00 GB


## Create artificial dataset

First we create an artificial dataset and write it to many CSV files.

You don't need to understand this section, we're just creating a dataset for the rest of the notebook.

In [11]:
import dask
df = dask.datasets.timeseries()
df

,id,name,x,y
npartitions=30,,,,
2000-01-01,int64,object,float64,float64
2000-01-02,...,...,...,...
...,...,...,...,...
2000-01-30,...,...,...,...
2000-01-31,...,...,...,...


In [12]:
import os
import datetime

if not os.path.exists('data'):
    os.mkdir('data')

def name(i):
    """ Provide date for filename given index
    
    Examples
    --------
    >>> name(0)
    '2000-01-01'
    >>> name(10)
    '2000-01-11'
    """
    return str(datetime.date(2000, 1, 1) + i * datetime.timedelta(days=1))
    
df.to_csv('data/*.csv', name_function=name);

## Read CSV files

We now have many CSV files in our data directory, one for each day in the month of January 2000.  Each CSV file holds timeseries data for that day.  We can read all of them as one logical dataframe using the `dd.read_csv` function with a glob string.

In [13]:
!ls data/*.csv | head

data/2000-01-01.csv
data/2000-01-02.csv
data/2000-01-03.csv
data/2000-01-04.csv
data/2000-01-05.csv
data/2000-01-06.csv
data/2000-01-07.csv
data/2000-01-08.csv
data/2000-01-09.csv
data/2000-01-10.csv


In [14]:
!head data/2000-01-01.csv

timestamp,id,name,x,y
2000-01-01 00:00:00,983,Tim,0.7492707897015314,0.2648111158151951
2000-01-01 00:00:01,1036,Victor,-0.7827738360518977,0.9525866786060084
2000-01-01 00:00:02,999,Zelda,0.5752441323952828,-0.20440291147207534
2000-01-01 00:00:03,995,Edith,-0.6216401742158022,0.2802615886207278
2000-01-01 00:00:04,990,Hannah,0.6238838308272716,0.8136081887819291
2000-01-01 00:00:05,989,George,-0.8378066407685931,0.681429179437611
2000-01-01 00:00:06,962,Zelda,-0.21110316826134778,0.774692628624855
2000-01-01 00:00:07,968,Charlie,0.0441103447536364,-0.19279839166355117
2000-01-01 00:00:08,1000,Patricia,0.446528750453983,-0.8364031154743485


In [15]:
!head data/2000-01-30.csv

timestamp,id,name,x,y
2000-01-30 00:00:00,1005,Kevin,-0.9959316227866193,-0.2090838634569112
2000-01-30 00:00:01,1026,Wendy,-0.8176236587051329,-0.5710017898095643
2000-01-30 00:00:02,985,Quinn,-0.12992180717345447,-0.5187081466761867
2000-01-30 00:00:03,958,George,0.7010104283049972,-0.09775827923753799
2000-01-30 00:00:04,991,Frank,0.9676984181306447,-0.982685532150066
2000-01-30 00:00:05,982,Wendy,-0.46834776044652116,-0.36126412813728215
2000-01-30 00:00:06,965,Charlie,-0.3617143821292206,0.1941353231890912
2000-01-30 00:00:07,1013,Charlie,0.8843225405660344,-0.5175110499045088
2000-01-30 00:00:08,980,Quinn,0.00741997492062163,-0.44501121854640346


We can read one file with `pandas.read_csv` or many files with `dask.dataframe.read_csv`

In [16]:
import pandas as pd

df = pd.read_csv('data/2000-01-01.csv')
df.head()

,timestamp,id,name,x,y
0,2000-01-01 00:00:00,983,Tim,0.749271,0.264811
1,2000-01-01 00:00:01,1036,Victor,-0.782774,0.952587
2,2000-01-01 00:00:02,999,Zelda,0.575244,-0.204403
3,2000-01-01 00:00:03,995,Edith,-0.621640,0.280262
4,2000-01-01 00:00:04,990,Hannah,0.623884,0.813608


In [17]:
import dask.dataframe as dd

df = dd.read_csv('data/2000-*-*.csv')
df

,timestamp,id,name,x,y
npartitions=30,,,,,
,object,int64,object,float64,float64
,...,...,...,...,...
...,...,...,...,...,...
,...,...,...,...,...
,...,...,...,...,...


In [18]:
df.head()

,timestamp,id,name,x,y
0,2000-01-01 00:00:00,983,Tim,0.749271,0.264811
1,2000-01-01 00:00:01,1036,Victor,-0.782774,0.952587
2,2000-01-01 00:00:02,999,Zelda,0.575244,-0.204403
3,2000-01-01 00:00:03,995,Edith,-0.621640,0.280262
4,2000-01-01 00:00:04,990,Hannah,0.623884,0.813608


## Tuning read_csv

The Pandas `read_csv` function has *many* options to help you parse files.  The Dask version uses the Pandas function internally, and so supports many of the same options.  You can use the `?` operator to see the full documentation string.

In [19]:
pd.read_csv?

Signature:
pd.read_csv(
    filepath_or_buffer: Union[str, pathlib.Path, IO[~AnyStr]],
    sep=',',
    delimiter=None,
    header='infer',
    names=None,
    index_col=None,
    usecols=None,
    squeeze=False,
    prefix=None,
    mangle_dupe_cols=True,
    dtype=None,
    engine=None,
    converters=None,
    true_values=None,
    false_values=None,
    skipinitialspace=False,
    skiprows=None,
    skipfooter=0,
    nrows=None,
    na_values=None,
    keep_default_na=True,
    na_filter=True,
    verbose=False,
    skip_blank_lines=True,
    parse_dates=False,
    infer_datetime_format=False,
    keep_date_col=False,
    date_parser=None,
    dayfirst=False,
    cache_dates=True,
    iterator=False,
    chunksize=None,
    compression='infer',
    thousands=None,
    decimal: str = '.',
    lineterminator=None,
    quotechar='"',
    quoting=0,
    doublequote=True,
    escapechar=None,
    comment=None,
    encoding=None,
    dialect=None,
    error_bad_lines=True,
    warn_bad_l

In [20]:
dd.read_csv?

Signature:
dd.read_csv(
    urlpath,
    blocksize='default',
    lineterminator=None,
    compression=None,
    sample=256000,
    enforce=False,
    assume_missing=False,
    storage_options=None,
    include_path_column=False,
    **kwargs,
)
Docstring:
Read CSV files into a Dask.DataFrame

This parallelizes the :func:`pandas.read_csv` function in the following ways:

- It supports loading many files at once using globstrings:

    >>> df = dd.read_csv('myfiles.*.csv')  # doctest: +SKIP

- In some cases it can break up large files:

    >>> df = dd.read_csv('largefile.csv', blocksize=25e6)  # 25MB chunks  # doctest: +SKIP

- It can read CSV files from external resources (e.g. S3, HDFS) by
  providing a URL:

    >>> df = dd.read_csv('s3://bucket/myfiles.*.csv')  # doctest: +SKIP
    >>> df = dd.read_csv('hdfs:///myfiles.*.csv')  # doctest: +SKIP
    >>> df = dd.read_csv('hdfs://namenode.example.com/myfiles.*.csv')  # doctest: +SKIP

Internally ``dd.read_csv`` uses :func:`pandas.read

In this case we use the `parse_dates` keyword to parse the timestamp column to be a datetime.  This will make things more efficient in the future.  Notice that the dtype of the timestamp column has changed from `object` to `datetime64[ns]`.

In [21]:
df = dd.read_csv('data/2000-*-*.csv', parse_dates=['timestamp'])
df

,timestamp,id,name,x,y
npartitions=30,,,,,
,datetime64[ns],int64,object,float64,float64
,...,...,...,...,...
...,...,...,...,...,...
,...,...,...,...,...
,...,...,...,...,...


## Do a simple computation

Whenever we operate on our dataframe we read through all of our CSV data so that we don't fill up RAM.  This is very efficient for memory use, but reading through all of the CSV files every time can be slow.

In [ ]:
%time df.groupby('name').x.mean().compute()

## Write to Parquet

Instead, we'll store our data in Parquet, a format that is more efficient for computers to read and write.

In [ ]:
df.to_parquet('data/2000-01.parquet', engine='pyarrow')

In [ ]:
!ls data/2000-01.parquet/

## Read from Parquet

In [ ]:
df = dd.read_parquet('data/2000-01.parquet', engine='pyarrow')
df

In [ ]:
%time df.groupby('name').x.mean().compute()

## Select only the columns that you plan to use

Parquet is a column-store, which means that it can efficiently pull out only a few columns from your dataset.  This is good because it helps to avoid unnecessary data loading.

In [ ]:
%%time
df = dd.read_parquet('data/2000-01.parquet', columns=['name', 'x'], engine='pyarrow')
df.groupby('name').x.mean().compute()

Here the difference is not that large, but with larger datasets this can save a great deal of time.

## Learn more

http://dask.pydata.org/en/latest/dataframe-create.html